# GET NEXT LINE
- Can use read, malloc, and free

## get_next_line.h

In [459]:
%%file ../gnl_working/get_next_line.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.h                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:29 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:32 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef GET_NEXT_LINE_H
# define GET_NEXT_LINE_H
# ifndef BUFFER_SIZE
#  define BUFFER_SIZE 1024
# endif
# include <stdlib.h>
# include <unistd.h>

typedef struct s_list
{
	char			*content;
	struct s_list	*next;
}					t_list;

char	*get_next_line(int fd);

#endif

Overwriting ../gnl_working/get_next_line.h


## ~M~A~N~D~A~T~O~R~Y~

### get_next_line.c

In [460]:
%%file ../gnl_working/get_next_line.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:18 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:23 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

char	*get_next_line(int fd)
{
	t_list	*first
	char	*buffer;
	char	*return_string;

	first = 0;
	buffer = (char *)malloc(BUFFER_SIZE + 1 * sizeof(char));
	if (!buffer)
		return (0);
	read(fd, buffer, BUFFER_SIZE);
	buffer[BUFFER_SIZE] = 0;
	while (!new_line_in_buffer(buffer))
	{
		add_chunk(&first, fd, buffer);
	}
	return_string = fill_return(first);
	if (!return_string)
		return (0);
	free(buffer);
	return (buffer);
}


Overwriting ../gnl_working/get_next_line.c


### get_next_line_utils.c

In [461]:
%%file ../gnl_working/get_next_line_utils.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_utils.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:38 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:41 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

static int	new_line_in_buffer(char *buffer)
{
	int	i;

	i = 0;
	while (buffer[i])
	{
		if (buffer[i] == '\n')
			return (i);
	}
	return (0);
}

static char	*fill_return(t_list *lst)
{
	t_list	*current;
	char	*ret;
	int		i;
	int		j;

	i = 0;
	current = lst;
	while (current->next)
	{
		current = current->next;
		i++;
	}
	j = 0;
	while (current->content[j])
		j++;
	ret = (char *)malloc(((BUFFER_SIZE * i) + j + 1) * sizeof(char));
	if (!ret)
		return (0);
	return (ret);
}

static char	*ft_strdup(const char *src)
{
	char	*str_ret;
	int		src_len;
	int		i;

	i = 0;
	src_len = 0;
	while (src[src_len])
		src_len++;
	str_ret = (char *)malloc(src_len + 1 * sizeof(char));
	if (!str_ret)
		return (0);
	while (src[i])
	{
		str_ret[i] = src[i];
		i++;
	}
	str_ret[i] = '\0';
	return (str_ret);
}

static t_list	*add_chunk(t_list **lst, int fd, char *buf)
{
	t_list	*new;
	t_list	*current;
	int		i;

	i = 0;
	new = (t_list *)malloc(sizeof(t_list));
	if (!new)
		return (0);
	new->content = ft_strdup(buf);
	if (!new->content)
		return (0);
	new->next = NULL;
	if (!*lst)
		*lst = new;
	else
	{
		current = *lst;
		while (current->next)
			current = current->next;
		current->next = new;
	}
	return (new);
}


Overwriting ../gnl_working/get_next_line_utils.c


### gnl_test

In [457]:
%%file gnl_test.c
#include "../gnl_working/get_next_line.h"
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>

int main (void)
{
	char	*string;
	int		fd;

	fd = open("output.csv", O_RDONLY);
	string = get_next_line(fd);
	printf("Return:%s\n", string);

	free(string);
	close(fd);

	return (0);
}

Overwriting gnl_test.c


In [458]:
%%bash
cc -Wall -Wextra -Werror -D BUFFER_SIZE=42 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -o gnl_test
./gnl_test
valgrind --leak-check=full ./gnl_test

Return:label,size,member type,member name,member 
Return:label,size,member type,member name,member 


==127071== Memcheck, a memory error detector
==127071== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==127071== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==127071== Command: ./gnl_test
==127071== 
==127071== 
==127071== HEAP SUMMARY:
==127071==     in use at exit: 0 bytes in 0 blocks
==127071==   total heap usage: 2 allocs, 2 frees, 4,139 bytes allocated
==127071== 
==127071== All heap blocks were freed -- no leaks are possible
==127071== 
==127071== For lists of detected and suppressed errors, rerun with: -s
==127071== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


## ~B~O~N~U~S~

- Add bonus material here

## Makefile

### Makefile

In [14]:
%%file ../gnl_testing/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2022/12/01 14:36:56 by dpentlan          #+#    #+#              #
#    Updated: 2022/12/01 14:36:58 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

# Not needed for get next line. But in case I want to use this later it's here.
# Note that is not working yet..

NAME = gnl

SRCS =	get_next_line_utils.c \
	get_next_line.c

BONUS_SRCS =

OBJS = ${SRCS:.c=.o}

BONUS_OBJS = ${BONUS_SRCS:.c=.o}

INCS = get_next_line.h

CFLAGS = -Wall -Wextra -Werror

CC = cc

SLIB = ar rcs

all: $(NAME)

$(NAME): $(OBJS) $(INCS)
	@$(SLIB) $(NAME) -o $(OBJS)

bonus: $(OBJS) $(BONUS_OBJS) $(INCS)
	@$(SLIB) $(NAME) -o $(OBJS) $(BONUS_OBJS)

%.o: %.c
	@$(CC) ${CFLAGS} -c $< -o ${<:.c=.o}

clean:
	@rm -f $(OBJS) $(BONUS_OBJS)

fclean:	clean
	@rm -f $(NAME)

re:	fclean all


Overwriting ../gnl_testing/Makefile


### Build Mandatory

In [ ]:
%%bash
cd ../gnl_working
# add build commands here

## Scratch

### File descriptors
- I found out after making this that the struct _IO_FILE is included in the standard library and is used to help interaction with files, but it is not used with read, open, write, close, etc. 
- It is used with fread, fopen, fwrite, and fclose.

In [268]:
%%file fd_examine.h
#ifndef FD_EXAMINE_H_
# define FD_EXAMINE_H_
# include <stdio.h>
# include <unistd.h>

void	print_file_struct_properties(char *label, FILE *p_file);

#endif

Overwriting fd_examine.h


In [377]:
%%file fd_examine.c
#include "fd_examine.h"

void	print_file_struct_properties(char *label, FILE *p_file)
{
	printf("label,size,member type,member name,member description,value,offset from beginning of struct\n");
	printf("whole struct,%ld,,,\n", sizeof(*p_file));
	printf("%s,%d,int,_flags,no description,%d,%ld\n", label, 8, stdin->_flags, (((unsigned long int) &(stdin->_flags)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_ptr,Current read pointer,%p,%ld\n", label, 8, stdin->_IO_read_ptr, (((unsigned long int) &(stdin->_IO_read_ptr)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_end,End of get area,%p,%ld\n", label, 8, stdin->_IO_read_end, (((unsigned long int) &(stdin->_IO_read_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_base,Start of putback+get area,%p,%ld\n", label, 8, stdin->_IO_read_base, (((unsigned long int) &(stdin->_IO_read_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_base,Start of put area,%p,%ld\n", label, 8, stdin->_IO_write_base, (((unsigned long int) &(stdin->_IO_write_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_ptr,Current put pointer,%p,%ld\n", label, 8, stdin->_IO_write_ptr, (((unsigned long int) &(stdin->_IO_write_ptr)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_end,End of put area,%p,%ld\n", label, 8, stdin->_IO_write_end, (((unsigned long int) &(stdin->_IO_write_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_buf_base,Start of reserve area,%p,%ld\n", label, 8, stdin->_IO_buf_base, (((unsigned long int) &(stdin->_IO_buf_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_buf_end,End of reserve area,%p,%ld\n", label, 8, stdin->_IO_buf_end, (((unsigned long int) &(stdin->_IO_buf_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_save_base,Pointer to start of non-current get area,%p,%ld\n", label, 8, stdin->_IO_save_base, (((unsigned long int) &(stdin->_IO_save_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_backup_base,Pointer to first valid character of backup area,%p,%ld\n", label, 8, stdin->_IO_backup_base, (((unsigned long int) &(stdin->_IO_backup_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_save_end,Pointer to end of non-current get area,%p,%ld\n", label, 8, stdin->_IO_save_end, (((unsigned long int) &(stdin->_IO_save_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,struct _IO_marker *,_markers,no description,%p,%ld\n", label, 8, stdin->_markers, (((unsigned long int) &(stdin->_markers)) - ((unsigned long int) stdin)));
	printf("%s,%d,struct _IO_FILE *,_chain,no description,%p,%ld\n", label, 8, stdin->_chain, (((unsigned long int) &(stdin->_chain)) - ((unsigned long int) stdin)));
	printf("%s,%d,int,_fileno,no description,%d,%ld\n", label, 4, stdin->_fileno, (((unsigned long int) &(stdin->_fileno)) - ((unsigned long int) stdin)));
	//printf("%s,%d,int,_blksize,no description,%d,%ld\n", label, 4, stdin->_blksize, (((unsigned long int) &(stdin->_blksize)) - ((unsigned long int) stdin)));
	printf("%s,%d,int,_flags2,no description,%d,%ld\n", label, 4, stdin->_flags2, (((unsigned long int) &(stdin->_flags2)) - ((unsigned long int) stdin)));
	printf("%s,%d,_IO_off_t,_old_offset,This used to be _offset but it's too small,%ld,%ld\n", label, 8, stdin->_old_offset, (((unsigned long int) &(stdin->_old_offset)) - ((unsigned long int) stdin)));
	printf("%s,%d,unsigned short,_cur_column,no description,%d,%ld\n", label, 2, stdin->_cur_column, (((unsigned long int) &(stdin->_cur_column)) - ((unsigned long int) stdin)));
	printf("%s,%d,signed char,_vtable_offset,no description,%d,%ld\n", label, 8, stdin->_vtable_offset, (((unsigned long int) &(stdin->_vtable_offset)) - ((unsigned long int) stdin)));
	printf("%s,%d,char,_shortbuf,no description,%s,%ld\n", label, 8, stdin->_shortbuf, (((unsigned long int) &(stdin->_shortbuf)) - ((unsigned long int) stdin)));
	printf("%s,%d,_IO_lock_t *,_lock,no description,%p,%ld\n", label, 8, stdin->_lock, (((unsigned long int) &(stdin->_lock)) - ((unsigned long int) stdin)));

	return ;
}

Overwriting fd_examine.c


In [380]:
%%file fd_test.c
#include <stdio.h>
#include <unistd.h>
#include "fd_examine.h"

int main (void)
{
	//printf("i/o    :descriptor\tpointer\t\t\n");
	//printf("stdin  :%d\t\t%p\n", STDIN_FILENO, stdin);
	//printf("stdout :%d\t\t%p\n", STDOUT_FILENO, stdout);
	// printf("stderr :%d\t\t%p\n", STDERR_FILENO, stderr);

	print_file_struct_properties("stdin", stdin);
	//print_file_struct_properties("stdout", stdout);
	//print_file_struct_properties("stderr", stderr);

	return (0);
}

Overwriting fd_test.c


In [384]:
%%bash
gcc fd_test.c fd_examine.c
./a.out > output.csv